In [10]:
from utils import *

from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

In [1]:
# Rutas de los datos
ruta_entrenamiento = "db/KDDTrain+.arff"
ruta_prueba = "db/KDDTest+.arff"
ruta_entrenamiento, ruta_prueba

('db/KDDTrain+.arff', 'db/KDDTest+.arff')

In [4]:
X_entrenamiento, y_entrenamiento = procesar_datos_entrada(ruta_entrenamiento)
X_prueba, y_prueba = procesar_datos_entrada(ruta_prueba)

print("Dimensiones de las características de entrenamiento:",
    X_entrenamiento.shape)
print("Dimensiones de las características de prueba:", X_prueba.shape)

Dimensiones de las características de entrenamiento: (400, 41)
Dimensiones de las características de prueba: (400, 41)


In [6]:
preprocesador = obtener_preprocesador(X_entrenamiento)
preprocesador

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 Index(['count', 'diff_srv_rate', 'dst_bytes', 'dst_host_count',
       'dst_host_diff_srv_rate', 'dst_host_rerror_rate',
       'dst_host_same_src_port_rate', 'dst_host_same_srv_rate',
       'dst_host_serror_rate', 'dst_host_srv_count',
       'dst_host_srv_diff_host_rate', 'dst_host_srv_rerror_rate',
       'dst_host_srv_serror_rate', 'du...
       'num_outbound_cmds', 'num_root', 'num_shells', 'rerror_rate',
       'root_shell', 'same_srv_rate', 'serror_rate', 'src_bytes', 'srv_count',
       'srv_diff_host_rate', 'srv_rerror_rate', 'srv_serror_rate',
       'su_attempted', 'urgent', 'wrong_fragment'],
      dtype='object')),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 Index(['protocol_type', 'service', 'flag', 'land', 'logged_in',
       'is_host_login', 'is_guest_login'],
      dtype='object'))])

In [12]:
k = len(y_entrenamiento.unique())

pipeline = Pipeline(steps=[
        ('preprocesador', preprocesador),
        ('clasificador', KMeans(n_clusters=k, random_state=42))
    ])

pipeline.fit(X_entrenamiento)

Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['count', 'diff_srv_rate', 'dst_bytes', 'dst_host_count',
       'dst_host_diff_srv_rate', 'dst_host_rerror_rate',
       'dst_host_same_src_port_rate', 'dst_host_same_srv_rate',
       'dst_host_serror_rate', 'dst_host_srv_count',
       'dst_host_srv_diff_host_rate', 'dst_host_srv_rerror_rat...
       'root_shell', 'same_srv_rate', 'serror_rate', 'src_bytes', 'srv_count',
       'srv_diff_host_rate', 'srv_rerror_rate', 'srv_serror_rate',
       'su_attempted', 'urgent', 'wrong_fragment'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['protocol_type', 'service', 'flag', 'land', 'logged_in',
       'is_host_login', 'is_guest_login'],
      dtype='object'))])),
                ('clasificador', KMeans(n_clusters=2, random_state=42))])

In [13]:
pipeline.labels_

AttributeError: 'Pipeline' object has no attribute 'labels_'